<a href="https://colab.research.google.com/github/HussainBadreddeen/AutoML_Thesis/blob/main/new_env_for_TPOT_and_H2o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')
# df_normalized_reviews.to_csv("/content/drive/My Drive/processed_data.csv", index=False, encoding="utf-8-sig") #fixed it utf-8 doesnt wokr although it did before
import pandas as pd
import numpy as np
#load it back
df_normalized_reviews2 = pd.read_csv("/content/drive/My Drive/processed_data.csv") #the nan row comes back somehow


Mounted at /content/drive


In [ ]:

print(df_normalized_reviews2[df_normalized_reviews2.isna().any(axis=1)])
empty_or_nan_values = df_normalized_reviews2[df_normalized_reviews2.isna().any(axis=1)]
empty_or_nan_values

df_normalized_reviews2 = df_normalized_reviews2[df_normalized_reviews2['stemmed_text'].notna()]  # remove rows where stemmed_text is NaN
print(df_normalized_reviews2[df_normalized_reviews2.isna().any(axis=1)])
empty_or_nan_values = df_normalized_reviews2[df_normalized_reviews2.isna().any(axis=1)]
empty_or_nan_values
#i saved the dataset again after here and now it doesnt need removing this line again so no need to run this cell anymore

       rating  review normalized_review tokens filtered_tokens stemmed_text
59969       2  “ط. .                  ط  ['ط']              []          NaN
Empty DataFrame
Columns: [rating, review, normalized_review, tokens, filtered_tokens, stemmed_text]
Index: []


,rating,review,normalized_review,tokens,filtered_tokens,stemmed_text


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer # this is good: https://www.youtube.com/watch?v=rcovF6S1oFI
tfidf_vectorizer = TfidfVectorizer(
     # max_features=10000,      # Limit to 10k features # update i will use just min and max and wont test chunking on all 71k unique words/features
    min_df=5,                # Remove rare words (appearing in less than 5 documents/rows)
    max_df=0.8,              # Remove overly common words (appearing in more than 80% of documents)
    dtype=np.float32          # Use float32 to save memory
)


# Fit and transform the stemmed text vocab
tfidf_matrix = tfidf_vectorizer.fit_transform(df_normalized_reviews2["stemmed_text"])

# Print the shape of the matrix
print("Shape of the TF-IDF matrix:", tfidf_matrix.shape)
#converting the sparse matrix to dense in case i need to use the dense matrix
X_dense = tfidf_matrix.astype(np.float32).toarray()

np.save('C:/Users/hussa/OneDrive/Desktop/thesis_code/tfidf_dense.npy', X_dense)


Shape of the TF-IDF matrix: (102961, 9819)


#update using 71k words seems unprofessional might overfit and introduce noise and computation problems as im having them already.

#will probably use min_df and max_df to limit words more and try dense matrix again on drive first then maybe local but will always opt for drive till exhaustion

#loading dense matrix and ratings from drive

In [ ]:
import numpy as np

# Load pre-saved files #the labels are already mapped and the dense matrix is already ready and is 3.7gb (9.8k words converted using float32 isntead of 64) insteado of 55gb with all words
X = np.load('C:/Users/hussa/OneDrive/Desktop/thesis_code/tfidf_dense.npy', mmap_mode='r')  # efficient read
y = np.load('C:/Users/hussa/OneDrive/Desktop/thesis_code/labels_binary.npy')

# H2o attempt in running with dense matrix and mapped ratings

In [ ]:
import h2o
from h2o.automl import H2OAutoML
import numpy as np

# Load pre-saved files # i do that up incase i need to use with diff models or i can just load it again for each model

from google.colab import drive
drive.mount('/content/drive')
X = np.load('/content/drive/My Drive/tfidf_dense.npy', mmap_mode='r')  # efficient read
y = np.load('/content/drive/My Drive/labels_binary.npy')

# Initialize H2O
h2o.init()

# Convert the dense numpy array X to an H2OFrame
X_h2o = h2o.H2OFrame(X)

# Convert the numpy array y to an H2OFrame
y_h2o = h2o.H2OFrame(y, column_names=['rating']) # Assuming 'labels_binary' corresponds to the rating

# Combine features (X) and target (y) into a single H2OFrame
df_h2o = X_h2o.cbind(y_h2o)

# Specify the target column and features
target_column = 'rating'
feature_columns = df_h2o.columns
feature_columns.remove(target_column)

# Split the data into training and testing sets (use H2O's split_frame method)
train, test = df_h2o.split_frame(ratios=[.8], seed=1) #seed =1 to ensure that the random operations (like the train-test split or cross-validation splits) produce the same results each time the code is run so we can reproduce same results

#if you run the same code with the same seed, you’ll get the same results every time.


# Run AutoML with the training data
aml = H2OAutoML(max_models=10, seed=1, nfolds=5,max_runtime_secs=3600)  # You can adjust parameters as needed
aml.train(x=feature_columns, y=target_column, training_frame=train)

# View the leaderboard (top-performing models)
lb = aml.leaderboard
print(lb)

# Predict on the test set
preds = aml.predict(test)

# Evaluate the predictions
print(preds)

# Shutdown the H2O cluster
h2o.cluster().shutdown()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.26" 2025-01-21; OpenJDK Runtime Environment (build 11.0.26+4-post-Ubuntu-1ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.26+4-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.11/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpl7f4q5xd
  JVM stdout: /tmp/tmpl7f4q5xd/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpl7f4q5xd/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_from_python_unknownUser_ihngge
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
!nvidia-smi
